# Capstone Project by Jayaprakash Saththasivam

#### Import all the required libraries

In [241]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library


#### Define a function to obtain the Latitude and Longitude for each District

In [242]:
def LocationName(location1):
    geolocator1 = Nominatim(user_agent='openmoves.net')
    try:
        location1 = geolocator1.geocode(location1)
    except:
        raise Exception("There was a problem with the geolocator function")
    return location1.latitude , location1.longitude

#### Our client has requested for a suitable location in Kuala Lumpur, Malaysia to expand their cafe in South East Asia. As the client is currently operating a very successful case at Marble Hill, Manhattan, we decided to explore the nearby venues in Marble Hill, Manhattan. The goal of this project is to identify a district in Kuala Lumpur with similar venues as Marble Hill. 

#### Lets first identify the location of Marble Hill, Manhanttan

In [243]:
address = "Marble Hill"
LatMH=LocationName(address)[0]
LongMH=LocationName(address)[1]
print('The geograpical coordinate of {} is {}°N, {}°W'.format(address, LatMH, -1*LongMH))


The geograpical coordinate of Marble Hill is 40.8762983°N, 73.9104292°W


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 1500 meters using Foursquare

#### Define the credentials of foursquare. As these contains my personal credentials, I have saved and read my credentials from a text file as this is a safe practise when publishing the code in public website 

In [244]:
f = open("foursquare_credentials.txt", "r")
CLIENT_ID = f.readline() # my Foursquare ID
CLIENT_SECRET = f.readline() # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
CLIENT_ID
f.close()


#### Let's create the GET request URL

In [245]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    LatMH, 
    LongMH, 
    radius, 
    LIMIT)

#### We obtain the results related to the venues of Marble Hill by sending a get request 

In [246]:
results = requests.get(url).json()

In [247]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [248]:
#venues = results['response']['groups'][0]['items']
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.rename(columns = {'venue.name':'Venue', 'venue.categories':'Venue Category', 
                              'venue.location.lat':'Venue Latitude', 'venue.location.lng':'Venue Longitude'}, inplace = True) 
   
nearby_venues.head()


,Venue,Venue Category,Venue Latitude,Venue Longitude
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Tibbett Diner,Diner,40.880404,-73.908937
2,Arturo's,Pizza Place,40.874412,-73.910271
3,Sam's Pizza,Pizza Place,40.879435,-73.905859
4,El Malecon,Caribbean Restaurant,40.879338,-73.904457


In [249]:
print('There are {} venues in Marble Hill.'.format(nearby_venues.count()[1]))
print('There are {} uniques categories in Marble Hill.'.format(len(nearby_venues['Venue Category'].unique())))

There are 100 venues in Marble Hill.
There are 51 uniques categories in Marble Hill.


#### Find the top ten venue categories in Marble Hill

In [250]:
top10marblehill = nearby_venues['Venue Category'].value_counts()/ len(nearby_venues['Venue Category'])
top10marblehill.head(10)

Pizza Place           0.07
Mexican Restaurant    0.07
Park                  0.04
Bakery                0.04
Café                  0.04
Coffee Shop           0.04
Bar                   0.04
Diner                 0.04
Wine Shop             0.03
Deli / Bodega         0.03
Name: Venue Category, dtype: float64

In [251]:
# create map of Marble Hill using latitude and longitude values
map_marblehill = folium.Map(location=[LatMH, LongMH], zoom_start=15)

# add markers to map
for lat, long, name, cat in zip(nearby_venues['Venue Latitude'], nearby_venues['Venue Longitude'], nearby_venues['Venue'], nearby_venues['Venue Category']):
    label = "Name: {} -- Category:{}".format(name, cat)   
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=7,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_marblehill) 
    
map_marblehill

### If the Folium map is not showing in the git hub, kindly use nbviewer to view this notebook

## Now Lets Investigate the Proposed Location

### Location of interest is Kuala Lumpur, capital of Malaysia

In [252]:
address = "Kuala Lumpur"
LatKL=LocationName(address)[0]
LongKL=LocationName(address)[1]
print('The geograpical coordinate of {} is {}°N, {} °E'.format(address, LatKL, LongKL))


The geograpical coordinate of Kuala Lumpur is 3.1516964°N, 101.6942371 °E


#### List of districts in Kuala Lumpur is not available online, hence I have created my own list. There are eleven districts in Kuala Lumpur (KL).

In [253]:

KL_district_list = ["Bukit Bintang", "Titiwangsa", "Setiawangsa", "Wangsa Maju","Batu, Kl", "Kepong", "Segambut","Lembah Pantai","Seputeh","Bandar Tun Razak","Cheras"]
df = pd.DataFrame(KL_district_list, columns=["District"]) 
df 


,District
0,Bukit Bintang
1,Titiwangsa
2,Setiawangsa
3,Wangsa Maju
4,"Batu, Kl"
5,Kepong
6,Segambut
7,Lembah Pantai
8,Seputeh
9,Bandar Tun Razak


In [254]:
df = pd.DataFrame(KL_district_list, columns=["District"]) 
df     

,District
0,Bukit Bintang
1,Titiwangsa
2,Setiawangsa
3,Wangsa Maju
4,"Batu, Kl"
5,Kepong
6,Segambut
7,Lembah Pantai
8,Seputeh
9,Bandar Tun Razak


### For Loop to obtain the coordinate for each district from geopy function

In [255]:
lat=[]
long=[]
sequence=df["District"]
for i in range(len(sequence)):
    lat1=LocationName((sequence[i]))[0]
    long1=LocationName((sequence[i]))[1]
    lat.append(lat1)
    long.append(long1)

In [256]:
df["Latitude"] = lat  
df["Longitude"]= long
df  

,District,Latitude,Longitude
0,Bukit Bintang,3.147107,101.708601
1,Titiwangsa,3.173145,101.695933
2,Setiawangsa,3.175725,101.735884
3,Wangsa Maju,3.205667,101.731908
4,"Batu, Kl",3.201823,101.671022
5,Kepong,3.205933,101.623711
6,Segambut,3.186437,101.664205
7,Lembah Pantai,3.104444,101.672189
8,Seputeh,3.113687,101.681420
9,Bandar Tun Razak,3.089695,101.712467


In [257]:
# create map of Kuala Lumpur using latitude and longitude values
map_KL = folium.Map(location=[LatKL, LongKL], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_KL)  
    
map_KL

### If the Folium map is not showing in the git hub, kindly use nbviewer to view this notebook

#### Explore the eleven districts in Kuala Lumpur using Foursquare

#### Let's create a function to explore the nearby venues in all the eleven districts in Kuala Lumpur

In [372]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called KL_venues.

In [373]:

KL_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bukit Bintang
Titiwangsa
Setiawangsa
Wangsa Maju
Batu, Kl
Kepong
Segambut
Lembah Pantai
Seputeh
Bandar Tun Razak
Cheras


In [374]:
print(KL_venues.shape)
KL_venues.head()

(1040, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bukit Bintang,3.147107,101.708601,Halab Arabic Cuisine,3.147739,101.708527,Middle Eastern Restaurant
1,Bukit Bintang,3.147107,101.708601,The KL Journal,3.148033,101.710645,Hotel
2,Bukit Bintang,3.147107,101.708601,良心 Liang Xin Relax Spa,3.144847,101.708914,Spa
3,Bukit Bintang,3.147107,101.708601,Pinchos,3.147963,101.707099,Tapas Restaurant
4,Bukit Bintang,3.147107,101.708601,Dining In The Dark KL,3.147408,101.708274,Restaurant


#### Let's check how many venues were returned for each district

In [375]:
KL_venues.groupby('District').count()
VenueCatperDis=KL_venues.groupby('District').count()
VenueCatperDis[["Venue"]]

,Venue
District,
Bandar Tun Razak,100
"Batu, Kl",60
Bukit Bintang,100
Cheras,92
Kepong,88
Lembah Pantai,100
Segambut,100
Seputeh,100
Setiawangsa,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [376]:
print('There are a total of {} venues in Kuala Lumpur.'.format(VenueCatperDis['Venue Category'].sum()))
print('There are {} uniques categories.'.format(len(KL_venues['Venue Category'].unique())))

There are a total of 1040 venues in Kuala Lumpur.
There are 188 uniques categories.


####  Analyze Each District

In [377]:
# one hot encoding
KL_onehot = pd.get_dummies(KL_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column back to dataframe
KL_onehot['District'] = KL_venues['District'] 

# move District column to the first column
fixed_columns = [KL_onehot.columns[-1]] + list(KL_onehot.columns[:-1])
KL_onehot = KL_onehot[fixed_columns]

KL_onehot.head()


,District,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Betting Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Cantonese Restaurant,Capitol Building,Chinese Restaurant,Clothing Store,Coffee Shop,College Academic Building,Comfort Food Restaurant,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Gaming Cafe,Gas Station,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Health & Beauty Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Train Station,Turkish Restaurant,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Volleyball Court,Warehouse Store,Wings Joint,Yakitori Restaurant,Yoga Studio
0,Bukit Bintang,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bukit Bintang,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bukit Bintang,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bukit Bintang,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [378]:
KL_onehot.shape

(1040, 189)

#### Next, let's group rows by district and by taking the mean of the frequency of occurrence of each category


In [379]:
KL_grouped = KL_onehot.groupby('District').mean().reset_index()
KL_grouped


,District,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Betting Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Cantonese Restaurant,Capitol Building,Chinese Restaurant,Clothing Store,Coffee Shop,College Academic Building,Comfort Food Restaurant,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Gaming Cafe,Gas Station,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Health & Beauty Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Train Station,Turkish Restaurant,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Volleyball Court,Warehouse Store,Wings Joint,Yakitori Restaurant,Yoga Studio
0,Bandar Tun Razak,0.01000,0.00,0.00,0.01000,0.00,0.00,0.050000,0.00,0.01000,0.000000,0.000000,0.00000,0.00,0.01000,0.00,0.030000,0.00,0.000000,0.00,0.00000,0.00000,0.000000,0.000000,0.00,0.01,0.010000,0.020000,0.030000,0.000000,0.00,0.120000,0.00,0.000000,0.01000,0.00,0.040000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.01,0.00,0.00000,0.00,0.030000,0.020000,0.00000,0.01000,0.01000,0.020000,0.00,0.010000,0.00,0.00,0.00,0.000000,0.03,0.000000,0.00,0.000000,0.020000,0.01,0.01,0.00,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.00,0.000000,0.00,0.00,0.040000,0.030000,0.000000,0.00,0.01000,0.00,0.00,0.00,0.000000,0.01000,0.00,0.00,0.00,0.01000,0.00,0.00,0.050000,0.010000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.010000,0.00,0.00,0.00000,0.00,0.020000,0.00,0.010000,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.030000,0.00,0.00,0.00,0.01000,0.000000,0.00,0.00,0.00,0.000000,0.00000,0.020000,0.020000,0.00,0.01000,0.00,0.01000,0.010000,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.02000,0.01000,0.01000,0.000000,0.000000,0.00,0.01,0.00,0.00,0.010000,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.030000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.01000,0.000000,0.00,0.00,0.00,0.00,0.00000,

In [380]:
#### Let's print each district along with the top 10 most common venues
num_top_venues = 10

for hood in KL_grouped['District']:
    print("----"+hood+"----")
    temp = KL_grouped[KL_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bandar Tun Razak----
                   venue  freq
0     Chinese Restaurant  0.12
1       Malay Restaurant  0.05
2       Asian Restaurant  0.05
3      Indian Restaurant  0.04
4      Convenience Store  0.04
5                   Café  0.03
6   Fast Food Restaurant  0.03
7            Pizza Place  0.03
8  Indonesian Restaurant  0.03
9        Thai Restaurant  0.03


----Batu, Kl----
                venue  freq
0  Chinese Restaurant  0.30
1    Asian Restaurant  0.13
2    Malay Restaurant  0.12
3          Food Truck  0.07
4  Seafood Restaurant  0.07
5          Restaurant  0.05
6     Motorcycle Shop  0.03
7        Burger Joint  0.03
8        Noodle House  0.02
9      Sandwich Place  0.02


----Bukit Bintang----
                 venue  freq
0                Hotel  0.10
1                 Café  0.05
2                  Spa  0.04
3       Clothing Store  0.04
4  Japanese Restaurant  0.04
5                  Bar  0.04
6             Boutique  0.03
7          Coffee Shop  0.03
8        Jewelry Store

#### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.

In [381]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [382]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = KL_grouped['District']

for ind in np.arange(KL_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(KL_grouped.iloc[ind, :], num_top_venues)

District_venues_sorted.head(11)


,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bandar Tun Razak,Chinese Restaurant,Asian Restaurant,Malay Restaurant,Indian Restaurant,Convenience Store,Gas Station,Café,Thai Restaurant,Fast Food Restaurant,Indonesian Restaurant
1,"Batu, Kl",Chinese Restaurant,Asian Restaurant,Malay Restaurant,Seafood Restaurant,Food Truck,Restaurant,Burger Joint,Motorcycle Shop,Sandwich Place,Café
2,Bukit Bintang,Hotel,Café,Clothing Store,Bar,Spa,Japanese Restaurant,Boutique,Coffee Shop,Shopping Mall,Malay Restaurant
3,Cheras,Chinese Restaurant,Malay Restaurant,Asian Restaurant,Indian Restaurant,Convenience Store,Indonesian Restaurant,Pizza Place,Italian Restaurant,Food Truck,Seafood Restaurant
4,Kepong,Chinese Restaurant,Café,Noodle House,Asian Restaurant,Restaurant,Food Truck,Coffee Shop,Fast Food Restaurant,Dessert Shop,Park
5,Lembah Pantai,Japanese Restaurant,Café,Malay Restaurant,Chinese Restaurant,Coffee Shop,Asian Restaurant,Restaurant,Pet Store,Fast Food Restaurant,Convenience Store
6,Segambut,Chinese Restaurant,Korean Restaurant,Malay Restaurant,Convenience Store,Coffee Shop,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Noodle House,Indian Restaurant
7,Seputeh,Japanese Restaurant,Ice Cream Shop,Café,Bakery,Clothing Store,Cosmetics Shop,Chinese Restaurant,Food Court,Department Store,Boutique
8,Setiawangsa,Malay Restaurant,Asian Restaurant,Thai Restaurant,Café,Bakery,Middle Eastern Restaurant,Food Truck,Park,Seafood Restaurant,Fast Food Restaurant
9,Titiwangsa,Malay Restaurant,Chinese Restaurant,Asian Restaurant,Café,Motorcycle Shop,Hotel,Thai Restaurant,Indian Restaurant,Coffee Shop,Clothing Store


#### Cluster District

In [383]:
# set number of clusters
kclusters = 4

KL_grouped_clustering = KL_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(KL_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 0, 1, 2, 0, 1, 2, 1, 3, 2])

In [385]:
# add clustering labels
District_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

KL_merged = df

# merge KL_grouped with KL_data to add latitude/longitude for each District
KL_merged = KL_merged.join(District_venues_sorted.set_index('District'), on='District')

KL_merged.head(11) 


,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bukit Bintang,3.147107,101.708601,1,Hotel,Café,Clothing Store,Bar,Spa,Japanese Restaurant,Boutique,Coffee Shop,Shopping Mall,Malay Restaurant
1,Titiwangsa,3.173145,101.695933,2,Malay Restaurant,Chinese Restaurant,Asian Restaurant,Café,Motorcycle Shop,Hotel,Thai Restaurant,Indian Restaurant,Coffee Shop,Clothing Store
2,Setiawangsa,3.175725,101.735884,3,Malay Restaurant,Asian Restaurant,Thai Restaurant,Café,Bakery,Middle Eastern Restaurant,Food Truck,Park,Seafood Restaurant,Fast Food Restaurant
3,Wangsa Maju,3.205667,101.731908,2,Chinese Restaurant,Asian Restaurant,Malay Restaurant,Thai Restaurant,Ice Cream Shop,Café,Burger Joint,Coffee Shop,Boutique,Convenience Store
4,"Batu, Kl",3.201823,101.671022,0,Chinese Restaurant,Asian Restaurant,Malay Restaurant,Seafood Restaurant,Food Truck,Restaurant,Burger Joint,Motorcycle Shop,Sandwich Place,Café
5,Kepong,3.205933,101.623711,0,Chinese Restaurant,Café,Noodle House,Asian Restaurant,Restaurant,Food Truck,Coffee Shop,Fast Food Restaurant,Dessert Shop,Park
6,Segambut,3.186437,101.664205,2,Chinese Restaurant,Korean Restaurant,Malay Restaurant,Convenience Store,Coffee Shop,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Noodle House,Indian Restaurant
7,Lembah Pantai,3.104444,101.672189,1,Japanese Restaurant,Café,Malay Restaurant,Chinese Restaurant,Coffee Shop,Asian Restaurant,Restaurant,Pet Store,Fast Food Restaurant,Convenience Store
8,Seputeh,3.113687,101.681420,1,Japanese Restaurant,Ice Cream Shop,Café,Bakery,Clothing Store,Cosmetics Shop,Chinese Restaurant,Food Court,Department Store,Boutique
9,Bandar Tun Razak,3.089695,101.712467,2,Chinese Restaurant,Asian Restaurant,Malay Restaurant,Indian Restaurant,Convenience Store,Gas Station,Café,Thai Restaurant,Fast Food Restaurant,Indonesian Restaurant


#### Finally, let's visualize the resulting clusters

In [387]:
map_clusters = folium.Map(location=[ LatKL, LongKL], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(KL_merged['Latitude'], KL_merged['Longitude'], KL_merged['District'], KL_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters


### If the Folium map is not showing in the git hub, kindly use nbviewer to view this notebook

### Examine Clusters

#### Cluster 0

In [388]:
KL_merged.loc[KL_merged['Cluster Labels'] == 0, KL_merged.columns[[0] + list(range(4, KL_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Batu, Kl",Chinese Restaurant,Asian Restaurant,Malay Restaurant,Seafood Restaurant,Food Truck,Restaurant,Burger Joint,Motorcycle Shop,Sandwich Place,Café
5,Kepong,Chinese Restaurant,Café,Noodle House,Asian Restaurant,Restaurant,Food Truck,Coffee Shop,Fast Food Restaurant,Dessert Shop,Park


#### Cluster 1

In [389]:
KL_merged.loc[KL_merged['Cluster Labels'] == 1, KL_merged.columns[[0] + list(range(4, KL_merged.shape[1]))]]


,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bukit Bintang,Hotel,Café,Clothing Store,Bar,Spa,Japanese Restaurant,Boutique,Coffee Shop,Shopping Mall,Malay Restaurant
7,Lembah Pantai,Japanese Restaurant,Café,Malay Restaurant,Chinese Restaurant,Coffee Shop,Asian Restaurant,Restaurant,Pet Store,Fast Food Restaurant,Convenience Store
8,Seputeh,Japanese Restaurant,Ice Cream Shop,Café,Bakery,Clothing Store,Cosmetics Shop,Chinese Restaurant,Food Court,Department Store,Boutique


#### Cluster 2

In [390]:
KL_merged.loc[KL_merged['Cluster Labels'] == 2, KL_merged.columns[[0] + list(range(4, KL_merged.shape[1]))]]


,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Titiwangsa,Malay Restaurant,Chinese Restaurant,Asian Restaurant,Café,Motorcycle Shop,Hotel,Thai Restaurant,Indian Restaurant,Coffee Shop,Clothing Store
3,Wangsa Maju,Chinese Restaurant,Asian Restaurant,Malay Restaurant,Thai Restaurant,Ice Cream Shop,Café,Burger Joint,Coffee Shop,Boutique,Convenience Store
6,Segambut,Chinese Restaurant,Korean Restaurant,Malay Restaurant,Convenience Store,Coffee Shop,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Noodle House,Indian Restaurant
9,Bandar Tun Razak,Chinese Restaurant,Asian Restaurant,Malay Restaurant,Indian Restaurant,Convenience Store,Gas Station,Café,Thai Restaurant,Fast Food Restaurant,Indonesian Restaurant
10,Cheras,Chinese Restaurant,Malay Restaurant,Asian Restaurant,Indian Restaurant,Convenience Store,Indonesian Restaurant,Pizza Place,Italian Restaurant,Food Truck,Seafood Restaurant


#### Cluster 3

In [392]:
KL_merged.loc[KL_merged['Cluster Labels'] == 3, KL_merged.columns[[0] + list(range(4, KL_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Setiawangsa,Malay Restaurant,Asian Restaurant,Thai Restaurant,Café,Bakery,Middle Eastern Restaurant,Food Truck,Park,Seafood Restaurant,Fast Food Restaurant
